In [2]:
import h5py
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.transform import Rotation as R
import manifpy

import os


In [41]:
h5f = h5py.File("data/30_minutes.h5", "r")

In [46]:
id = 1
raw_data = np.array(h5f[f"tracker{id}"]) # time, xyz q_wxyz
q_ws = raw_data[:, 3:4]
raw_data[:, 3:6] = raw_data[:, 4:7]
raw_data[:, 6:7] = q_ws #xyz q_xyzw
max_sp = np.array([5.0, 5.0, 5.0, np.pi, np.pi, np.pi])/980.0
max_sp = np.array([np.inf]*6)

process_data = []
raw_velocity = []
filtered_SE3 = None
alpha = 1.0
for t in range(raw_data.shape[0]):
    if t==0:
        filtered_SE3 = manifpy.SE3(raw_data[t])
    else:
        se3 = manifpy.SE3(raw_data[t]) - filtered_SE3
        se3_clipped = se3.coeffs_copy()

        se3_clipped = np.clip(se3_clipped, -max_sp, max_sp)
        se3_clipped = manifpy.SE3Tangent(se3_clipped)

        filtered_SE3 = filtered_SE3 + (se3_clipped*alpha)
        raw_velocity.append(se3_clipped.coeffs_copy()*alpha)
    
    process_data.append(np.hstack([filtered_SE3.translation(), filtered_SE3.quat()]))

process_data = np.vstack(process_data)
raw_velocity = np.vstack(raw_velocity)*980.0

In [47]:
%matplotlib qt 

start_time_step = 0
max_time_step = raw_velocity.shape[0]

time = np.arange(max_time_step) / 1000
index = np.arange(max_time_step) + start_time_step
plt.rcParams["figure.figsize"] = (20, 8)
plt.subplot(2,3,1)
plt.plot(time, raw_velocity[index, 0], 'b')
# plt.plot(time, process_data[index, 0], '--r')

plt.subplot(2,3,2)
plt.plot(time, raw_velocity[index, 1], 'b')
# plt.plot(time, process_data[index, 1], '--r')

plt.subplot(2,3,3)
plt.plot(time, raw_velocity[index, 2], 'b')
# plt.plot(time, process_data[index, 2], '--r')

plt.subplot(2,3,4)
plt.plot(time, raw_velocity[index, 3], 'b')
# plt.plot(time, rpy_mat_proccessed[:,0], '--r')

plt.subplot(2,3,5)
plt.plot(time, raw_velocity[index, 4], 'b')
# plt.plot(time, rpy_mat_proccessed[:,1], '--r')

plt.subplot(2,3,6)
plt.plot(time, raw_velocity[index, 5], 'b')
# plt.plot(time, rpy_mat_proccessed[:,2], '--r')

plt.show()
# plt.plot(data[:,0])

In [47]:
folder = "25_06_2021_16_37_13_"
rchest_df = pd.read_csv(f"../data/{folder}/chest_raw.csv")
rrhand_df = pd.read_csv(f"../data/{folder}/rhand_raw.csv")
rlhand_df = pd.read_csv(f"../data/{folder}/lhand_raw.csv")

In [49]:
key = "x"
id = 0
start = 10
end = rchest_df.shape[0]
times = np.arange(end - start)/2000.0

In [50]:
rdist = np.linalg.norm(rchest_df.to_numpy()[start:end,:3] - rrhand_df.to_numpy()[start:end,:3], axis=1)
ldist = np.linalg.norm(rchest_df.to_numpy()[start:end,:3] - rlhand_df.to_numpy()[start:end,:3], axis=1)

In [51]:
# %matplotlib qt
# fig, axes = plt.subplots(2, 1)
plt.plot(times, rchest_df[key][start:end])
# plt.plot(fchest_df["x"][start:end])
plt.plot(times, rchest_df["status"][start:end]*0.1)
plt.plot(times, rdist)
plt.plot(times, ldist)
plt.legend([f"raw {key}", "status"])
plt.grid(True)
# plt.savefig(f"../data/{folder}/chest_{key}.jpg")

# SkeletonKinematics DATA

In [3]:
# folder = "13_07_2021_19_13_04_"
# df = pd.read_csv(f"../../data/{folder}/lsq_joint_data_20200706_sdh_exp2.csv")
# folder = "14_07_2021_11_33_35_"

folder = "selected_data/20200706_sdh_exp2_bag"

algos = ["Huber", "LSQ"]
datas = ["raw", "filter", "close", "pure"]

dfs = []

for algo in algos:
    for data in datas:
        filename = f"../../data/{folder}/{algo}Kinect_joint_data_{data}.csv"
        if os.path.isfile(filename):
            df = pd.read_csv(filename)
            dfs.append(df)


In [8]:
%matplotlib qt
# sec
start_time = 0.0
end_time = 140.0

num_dfs = len(dfs)

indicies = []
for i in range(num_dfs):
        index = dfs[i].index[(dfs[i]["time(s)"] >= start_time) & (dfs[i]["time(s)"] <= end_time)]
        indicies.append(index)

keys = ["body", "neck", 
        "left_shoulder", "left_upper_arm" , "left_lower_arm", "left_hand", 
        "right_shoulder", "right_upper_arm" , "right_lower_arm", "right_hand"]

values = range(10)

body2id = dict(zip(keys, values))

key = "left_shoulder"
start_id = 3 * body2id[key] + 1

fig, axes = plt.subplots(nrows=3, ncols=1)

joints = ["Z", "Y", "X"]

plot_ids = [0, 3, 4, 5]
for i in range(3):
        for j in plot_ids:
                axes[i].plot(dfs[j]["time(s)"][indicies[j]], dfs[j][f"q{start_id + i}"][indicies[j]])
        axes[i].legend([f"{j}_algo_{joints[i]}" for j in plot_ids])
        axes[i].grid(True)

fig.suptitle(f"{key.upper()} Joint")




Text(0.5, 0.98, 'LEFT_SHOULDER Joint')